In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import spacy
import re
import string
from spacy.lang.en import English
import nltk

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/taylor-swifts-youtube-stats-symphony/taylor_swift_videos.csv


In [2]:
df = pd.read_csv(r"/kaggle/input/taylor-swifts-youtube-stats-symphony/taylor_swift_videos.csv")
df

,title,published_at,duration,view_count
0,Taylor Swift - You're Losing Me (From The Vault),2023-11-29T13:00:47Z,PT4M39S,7278886
1,"Rent on demand starting … you guessed it, Dece...",2023-11-27T20:21:45Z,PT1M1S,1050064
2,✨🫶 My name is Taylor and I was born in 1989 🫶✨,2023-10-31T20:08:57Z,PT19S,1196267
3,Taylor Swift - Cruel Summer (LP Giobbi Remix),2023-10-19T00:30:06Z,PT3M19S,696792
4,Taylor Swift - Cruel Summer (Live from Taylor ...,2023-10-19T00:30:25Z,PT3M55S,3437493
...,...,...,...,...
218,First fans with my CMT Award.,2008-06-26T21:55:43Z,PT11S,337626
219,TRL Recap!,2008-06-26T21:15:51Z,PT3M8S,742898
220,Wannabe,2008-06-26T21:02:39Z,PT3M45S,1217405
221,:-),2008-06-26T20:51:09Z,PT34S,805812


In [3]:
print("DataFrame Information:")
print("-" * 30)
print('DF Shape: ' ,df.shape)
print('number of Columns: ' ,len(df.columns))
print('number of Observations: ' ,len(df))
print('Number of values in df: ' , df.count().sum())
print('Total Number of Missing values in df: ' , df.isna().sum().sum())
print('percentage of Missing values : ' ,  "{:.2f}".format(df.isna().sum().sum()/df.count().sum() *100),'%')
print('Total Number of Duplicated records in df : ' , df.duplicated().sum().sum())
print('percentage of Duplicated values : ' ,  "{:.2f}".format(df.duplicated().sum().sum()/df.count().sum() *100),'%')

DataFrame Information:
------------------------------
DF Shape:  (223, 4)
number of Columns:  4
number of Observations:  223
Number of values in df:  892
Total Number of Missing values in df:  0
percentage of Missing values :  0.00 %
Total Number of Duplicated records in df :  0
percentage of Duplicated values :  0.00 %


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         223 non-null    object
 1   published_at  223 non-null    object
 2   duration      223 non-null    object
 3   view_count    223 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 7.1+ KB


In [5]:
df['published_at'] = pd.to_datetime(df['published_at'])
df['date'] = df['published_at'].dt.date
df['time'] = df['published_at'].dt.time
df

,title,published_at,duration,view_count,date,time
0,Taylor Swift - You're Losing Me (From The Vault),2023-11-29 13:00:47+00:00,PT4M39S,7278886,2023-11-29,13:00:47
1,"Rent on demand starting … you guessed it, Dece...",2023-11-27 20:21:45+00:00,PT1M1S,1050064,2023-11-27,20:21:45
2,✨🫶 My name is Taylor and I was born in 1989 🫶✨,2023-10-31 20:08:57+00:00,PT19S,1196267,2023-10-31,20:08:57
3,Taylor Swift - Cruel Summer (LP Giobbi Remix),2023-10-19 00:30:06+00:00,PT3M19S,696792,2023-10-19,00:30:06
4,Taylor Swift - Cruel Summer (Live from Taylor ...,2023-10-19 00:30:25+00:00,PT3M55S,3437493,2023-10-19,00:30:25
...,...,...,...,...,...,...
218,First fans with my CMT Award.,2008-06-26 21:55:43+00:00,PT11S,337626,2008-06-26,21:55:43
219,TRL Recap!,2008-06-26 21:15:51+00:00,PT3M8S,742898,2008-06-26,21:15:51
220,Wannabe,2008-06-26 21:02:39+00:00,PT3M45S,1217405,2008-06-26,21:02:39
221,:-),2008-06-26 20:51:09+00:00,PT34S,805812,2008-06-26,20:51:09


In [6]:
from datetime import time
def fromat_duration(value):
    s = value.strip('PT')
    s = s.split('M')
    if len(s) > 1:
        if s[1] != '':
            sec = s[1].strip('S')
        else:
            sec = 0
        min = s[0]
    else:
        min = '0'
        sec = s[0].strip('S')

    return time(minute= int(min) , second=int(sec))


df['duration'] = df['duration'].apply(fromat_duration)



    

In [7]:
df

,title,published_at,duration,view_count,date,time
0,Taylor Swift - You're Losing Me (From The Vault),2023-11-29 13:00:47+00:00,00:04:39,7278886,2023-11-29,13:00:47
1,"Rent on demand starting … you guessed it, Dece...",2023-11-27 20:21:45+00:00,00:01:01,1050064,2023-11-27,20:21:45
2,✨🫶 My name is Taylor and I was born in 1989 🫶✨,2023-10-31 20:08:57+00:00,00:00:19,1196267,2023-10-31,20:08:57
3,Taylor Swift - Cruel Summer (LP Giobbi Remix),2023-10-19 00:30:06+00:00,00:03:19,696792,2023-10-19,00:30:06
4,Taylor Swift - Cruel Summer (Live from Taylor ...,2023-10-19 00:30:25+00:00,00:03:55,3437493,2023-10-19,00:30:25
...,...,...,...,...,...,...
218,First fans with my CMT Award.,2008-06-26 21:55:43+00:00,00:00:11,337626,2008-06-26,21:55:43
219,TRL Recap!,2008-06-26 21:15:51+00:00,00:03:08,742898,2008-06-26,21:15:51
220,Wannabe,2008-06-26 21:02:39+00:00,00:03:45,1217405,2008-06-26,21:02:39
221,:-),2008-06-26 20:51:09+00:00,00:00:34,805812,2008-06-26,20:51:09


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   title         223 non-null    object             
 1   published_at  223 non-null    datetime64[ns, UTC]
 2   duration      223 non-null    object             
 3   view_count    223 non-null    int64              
 4   date          223 non-null    object             
 5   time          223 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(4)
memory usage: 10.6+ KB


In [9]:
def clean_text(x):
    text = re.sub('(\d+)','',x)   
    text = text.lower()
    
    x = text 
    text_without_puct = [t for t in x if t not in string.punctuation]
    text_without_puct = ''.join(text_without_puct)
    return text_without_puct

nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
from nltk.corpus import stopwords
stop = stopwords.words('english')



[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df

,title,published_at,duration,view_count,date,time
0,Taylor Swift - You're Losing Me (From The Vault),2023-11-29 13:00:47+00:00,00:04:39,7278886,2023-11-29,13:00:47
1,"Rent on demand starting … you guessed it, Dece...",2023-11-27 20:21:45+00:00,00:01:01,1050064,2023-11-27,20:21:45
2,✨🫶 My name is Taylor and I was born in 1989 🫶✨,2023-10-31 20:08:57+00:00,00:00:19,1196267,2023-10-31,20:08:57
3,Taylor Swift - Cruel Summer (LP Giobbi Remix),2023-10-19 00:30:06+00:00,00:03:19,696792,2023-10-19,00:30:06
4,Taylor Swift - Cruel Summer (Live from Taylor ...,2023-10-19 00:30:25+00:00,00:03:55,3437493,2023-10-19,00:30:25
...,...,...,...,...,...,...
218,First fans with my CMT Award.,2008-06-26 21:55:43+00:00,00:00:11,337626,2008-06-26,21:55:43
219,TRL Recap!,2008-06-26 21:15:51+00:00,00:03:08,742898,2008-06-26,21:15:51
220,Wannabe,2008-06-26 21:02:39+00:00,00:03:45,1217405,2008-06-26,21:02:39
221,:-),2008-06-26 20:51:09+00:00,00:00:34,805812,2008-06-26,20:51:09
